In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import time

In [2]:
url = "https://data.ca.gov/dataset/590188d5-8545-4c93-a9a0-e230f0db7290/resource/926fd08f-cc91-4828-af38-bd45de97f8c3/download/statewide_cases.csv"

In [3]:
df = pd.read_csv(url)

### Clean Up

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7265 entries, 0 to 7264
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   county               7265 non-null   object 
 1   totalcountconfirmed  7262 non-null   float64
 2   totalcountdeaths     7263 non-null   float64
 3   newcountconfirmed    7265 non-null   int64  
 4   newcountdeaths       7265 non-null   int64  
 5   date                 7265 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 340.7+ KB


In [5]:
df

,county,totalcountconfirmed,totalcountdeaths,newcountconfirmed,newcountdeaths,date
0,Santa Clara,151.0,6.0,151,6,2020-03-18
1,Santa Clara,183.0,8.0,32,2,2020-03-19
2,Santa Clara,246.0,8.0,63,0,2020-03-20
3,Santa Clara,269.0,10.0,23,2,2020-03-21
4,Santa Clara,284.0,13.0,15,3,2020-03-22
...,...,...,...,...,...,...
7260,Yolo,941.0,28.0,52,0,2020-07-13
7261,Yolo,966.0,29.0,25,1,2020-07-14
7262,Yolo,1002.0,29.0,36,0,2020-07-15
7263,Yolo,1034.0,30.0,32,1,2020-07-16


In [6]:
df.fillna(0, inplace = True)
df.drop(df[(df['county'] == 'Out Of Country')].index, inplace = True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7168 entries, 0 to 7264
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   county               7168 non-null   object 
 1   totalcountconfirmed  7168 non-null   float64
 2   totalcountdeaths     7168 non-null   float64
 3   newcountconfirmed    7168 non-null   int64  
 4   newcountdeaths       7168 non-null   int64  
 5   date                 7168 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 392.0+ KB


In [26]:
county_df = df[df.groupby('county').date.transform('max') == df['date']]
county_df.reset_index(drop = True)
# county_df.drop(columns='index')

,county,totalcountconfirmed,totalcountdeaths,newcountconfirmed,newcountdeaths,date
0,Santa Clara,7080.0,177.0,90,1,2020-07-17
1,San Mateo,4407.0,114.0,71,0,2020-07-17
2,Santa Barbara,5001.0,32.0,129,0,2020-07-17
3,Tuolumne,73.0,0.0,2,0,2020-07-17
4,Sierra,1.0,0.0,0,0,2020-07-17
5,Placer,1207.0,11.0,39,0,2020-07-17
6,San Luis Obispo,1190.0,6.0,44,0,2020-07-17
7,Solano,2452.0,31.0,80,0,2020-07-17
8,Monterey,2922.0,18.0,65,0,2020-07-17
9,Yuba,231.0,3.0,10,0,2020-07-17


### Load to SQL

In [27]:
rds_connection_string = "postgres:postgres@localhost:5432/covid-vaccine-plz"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [28]:
engine.table_names()

['citywide_table', 'statewide_table']

In [29]:
county_df.to_sql(name='statewide_table',con=engine,if_exists='append',index=False)

In [30]:
pd.read_sql_query('select* from statewide_table',con=engine).head()

,county,totalcountconfirmed,totalcountdeaths,newcountconfirmed,newcountdeaths,date
0,Santa Clara,7080,177,90,1,2020-07-17
1,San Mateo,4407,114,71,0,2020-07-17
2,Santa Barbara,5001,32,129,0,2020-07-17
3,Tuolumne,73,0,2,0,2020-07-17
4,Sierra,1,0,0,0,2020-07-17


In [31]:
df.to_csv("./CSV/statewide.csv", index=False)